## Preparation

### Preparation > Imports

In [145]:
''' IMPORTS '''
import numpy as np
import pandas as pd
import matplotlib as plot
import math 

from enum import Enum    

### Preparation > Classes

In [146]:
'''CLASSES '''
class Cols(Enum):
        StatusOfExistingCheckingAccount = 0,
        DurationInMonth = 1, 
        CreditHistory = 2, 
        Purpose = 3, 
        CreditAmount = 4 , 
        SavingsAccountBonds =5,
        PresentEmploymentSince =6 ,
        InstallmentRateInPercentageOfDisposableIncome =7 ,
        PersonalStatusAndSex = 8,
        OtherDebtorsOrGuarantors =9 ,
        PresentResidenceSince =10 ,
        Property =11 ,
        AgeInYears = 12,
        OtherInstallmentPlans =13 ,
        Housing = 14,
        NumberOfExistingCreditsAtThisBank =15 ,
        Job = 16,
        NumberOfPeopleBeingLiableToProvideMaintenanceFor =17 ,
        Telephone = 18,
        ForeignWorker =19 ,
        ClassOutcome = 20
        
class Outcomes(Enum):
        Good = 1,
        Bad = 2

### Preparation > Constances

In [147]:
'''CONSTANCES'''
# dict of all column names
COLS = {
        Cols.StatusOfExistingCheckingAccount : "Status of existing checking account" ,
        Cols.DurationInMonth:"Duration in month" , 
        Cols.CreditHistory: "Credit history", 
        Cols.Purpose:"Purpose" , 
        Cols.CreditAmount:  "Credit amount", 
        Cols.SavingsAccountBonds: "Savings account/bonds" ,
        Cols.PresentEmploymentSince:"Present employment since" ,
        Cols.InstallmentRateInPercentageOfDisposableIncome: "Installment rate in percentage of disposable income",
        Cols.PersonalStatusAndSex: "Personal status and sex",
        Cols.OtherDebtorsOrGuarantors:"Other debtors / guarantors", 
        Cols.PresentResidenceSince: "Present residence since",
        Cols.Property: "Property", 
        Cols.AgeInYears: "Age in years",
        Cols.OtherInstallmentPlans:"Other installment plans", 
        Cols.Housing: "Housing", 
        Cols.NumberOfExistingCreditsAtThisBank:"Number of existing credits at this bank",
        Cols.Job: "Job",
        Cols.NumberOfPeopleBeingLiableToProvideMaintenanceFor:"Number of people being liable to provide maintenance for", 
        Cols.Telephone: "Telephone",
        Cols.ForeignWorker:"Foreign worker",
        Cols.ClassOutcome: "Class/Outcome"
}

# model parameter
PERC_TRAIN = 0.67
PERC_TEST = 1-PERC_TRAIN
ITERATIONS = 500


### Preparation > Functions

In [168]:
'''FUNCTIONS'''
def plot_gaussian():
    # Create graph for class 1
    print("plot_gaussian")
    # Create graph for class 2

    
    
# WIP    
# to calculate un-normalized posteriors and do classification with respect to g1 and g2results
def do_classification(df_test, prior_good, muv_good, cov_good, prior_bad, muv_bad, cov_bad):

    decision_list = []  # to store classification results to calculate accuracy later.
    for x in df_test:
        pred_good = discriminant_func(x=x, prior=prior_good, mus=muv_good, cov=cov_good)
        pred_bad = discriminant_func(x, prior_bad, muv_bad, cov_bad)

        print(f'{pred_good} and {pred_bad}')
        
    return decision_list


# WIP
def discriminant_func(x, prior, mus, cov):
    vari = math.sqrt(cov)
    res = 0
    for i in mus:
        res = res + (-(math.log(2*np.pi)/2) - math.log(vari) - ((x-mus[i])**2)/(2*cov) + np.log(prior))

# to find maximum of the discriminant function results
def find_max_likelihood(g1, g2):
    likelihood_list = [g1, g2]
    return max(likelihood_list)

## Model

### Model > Load Data

In [161]:
df = pd.read_csv(
    "german.data", 
    sep=" ", 
    names=[
        COLS[Cols.StatusOfExistingCheckingAccount],
        COLS[Cols.DurationInMonth],
        COLS[Cols.CreditHistory],
        COLS[Cols.Purpose],
        COLS[Cols.CreditAmount],
        COLS[Cols.SavingsAccountBonds],
        COLS[Cols.PresentEmploymentSince],
        COLS[Cols.InstallmentRateInPercentageOfDisposableIncome],
        COLS[Cols.PersonalStatusAndSex],
        COLS[Cols.OtherDebtorsOrGuarantors],
        COLS[Cols.PresentResidenceSince],
        COLS[Cols.Property],
        COLS[Cols.AgeInYears],
        COLS[Cols.OtherInstallmentPlans],
        COLS[Cols.Housing],
        COLS[Cols.NumberOfExistingCreditsAtThisBank],
        COLS[Cols.Job],
        COLS[Cols.NumberOfPeopleBeingLiableToProvideMaintenanceFor],
        COLS[Cols.Telephone],
        COLS[Cols.ForeignWorker],
        COLS[Cols.ClassOutcome],
    ])
df.head(1)

,Status of existing checking account,Duration in month,Credit history,Purpose,Credit amount,Savings account/bonds,Present employment since,Installment rate in percentage of disposable income,Personal status and sex,Other debtors / guarantors,...,Property,Age in years,Other installment plans,Housing,Number of existing credits at this bank,Job,Number of people being liable to provide maintenance for,Telephone,Foreign worker,Class/Outcome
0,A11,6,A34,A43,1169,A65,A75,4,A93,A101,...,A121,67,A143,A152,2,A173,1,A192,A201,1


## Model > Training (run  500x)

In [167]:
accuracy = 0  # total accuracy
attr1 = COLS[Cols.AgeInYears]
attr2 = COLS[Cols.CreditAmount]

# program will run for 500 times to find average accuracy
accuracies=[] # to store individual accuracies
for i in range(0, 1):
    # split data train & test
    msk = np.random.rand(len(df)) < PERC_TRAIN
    df_train = df[msk]
    df_test = df[~msk]
    print(f'len(train)={len(train)} and len(test)={len(test)}')
    
    # separate in class 1 & 2
    df_good = df_train.loc[df_train[COLS[Cols.ClassOutcome]] == 1]    
    df_bad  = df_train.loc[df_train[COLS[Cols.ClassOutcome]] == 2]
    
    prior_good = len(df_good)/(len(df_good)+len(df_bad)) 
    prior_bad = len(df_bad)/(len(df_good)+len(df_bad)) 
    
    # calculate MUs and COVs(MLE parameters) of each class
    muv_good = [df_good[attr1].mean(), df_good[attr2].mean()]
    cov_good = df_good[attr1].cov(df_good[attr2])
    
    muv_bad = [df_bad[attr1].mean(), df_bad[attr2].mean()]
    cov_bad = df_bad[attr1].cov(df_bad[attr2])

    
    # classification_result is an array that stores the result of classification
    classification_result = do_classification(
        df_test=df_test, 
        prior_good=prior_good, muv_good=muv_good, cov_good=cov_good, 
        prior_bad=prior_bad, muv_bad=muv_bad, cov_bad=cov_bad)
    

len(train)=646 and len(test)=354


ValueError: math domain error

In [151]:
    
 
'''
    
    # compare classification results with y_test
    individual_accuracy = accuracy_score(y_test, classification_result)
    accuracy = accuracy + individual_accuracy
    accuracies.append(individual_accuracy)

print("Average Accuracy = %", (accuracy/ITERATIONS)*100)
print("Max Accuracy = %", max(accuracies)*100)

plot_gaussian()

'''

'\n    \n    # compare classification results with y_test\n    individual_accuracy = accuracy_score(y_test, classification_result)\n    accuracy = accuracy + individual_accuracy\n    accuracies.append(individual_accuracy)\n\nprint("Average Accuracy = %", (accuracy/ITERATIONS)*100)\nprint("Max Accuracy = %", max(accuracies)*100)\n\nplot_gaussian()\n\n'